In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
#import tensorflow as tf
from scipy import stats
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
#from keras import backend as K
from sklearn.metrics import mean_absolute_error
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
sns.set()
%matplotlib inline

In [ ]:
raw_data=pd.read_csv('training-data.csv')
raw_data

In [ ]:
raw_data['Appliances'].describe()

In [ ]:
print("Skewness : %f" % raw_data['Appliances'].skew())
print("Kurtosis : %f" % raw_data['Appliances'].kurt())

In [ ]:
#We need to check info() to know if there's any missing values
raw_data.info()

In [ ]:
sns.distplot(raw_data.loc[:, 'RH_5'])

In [ ]:
log_rh5 = raw_data[['RH_5']].applymap(lambda x: np.log(x+1))
log_rh5

In [ ]:
sns.distplot(log_rh5)

In [ ]:
sns.distplot(raw_data.loc[:, 'Windspeed'])

In [ ]:
log_windspeed = raw_data[['Windspeed']].applymap(lambda x: np.log(x+1))
log_windspeed.describe()

In [ ]:
copy_data = pd.DataFrame.copy(raw_data)

In [ ]:
copy_data

# Remove Features rv1 & rv2

In [ ]:
new_copy_data=copy_data.copy()
new_copy_data=new_copy_data.drop(['rv1','rv2'],axis=1)
new_copy_data

# Using Multi-layer Perceptron regressor.

In [ ]:
model= MLPRegressor(
    hidden_layer_sizes=(19,),
    activation='relu',
    solver='adam',
    alpha=0.0001,
    batch_size=256,
    learning_rate='constant',
    learning_rate_init=0.001,
    power_t=0.5,
    momentum=0.9,
    verbose=1,
    early_stopping=True)

In [ ]:
X_train=new_copy_data.loc[:, :'Tdewpoint']
X_train

'''X_train=raw_data.loc[:, :'rv2']
X_train'''

In [ ]:
Y_train=new_copy_data.loc[:,'Appliances':]
Y_train

'''Y_train=raw_data.loc[:,'Appliances':]
Y_train'''

In [ ]:
model.fit(X_train,Y_train)

# Predict the Appliances and Lights from data_testing 

In [ ]:
newtestFeature=pd.read_csv('testing-data.csv')
newtestFeature=newtestFeature.drop(['rv1','rv2'],axis=1)
newtestFeature

In [ ]:
model.out_activation_='relu'
predictions=model.predict(newtestFeature)
predictionsDataFrame=pd.DataFrame({'Appliances':predictions[:,0],'Lights':predictions[:,1]})
join=newtestFeature.join(predictionsDataFrame)
join

# Creating Mean Absolute Error

In [ ]:
#Take Index Appliances and Lights from data_training
y_true=np.array(copy_data[['Appliances','lights']])

#Predict the Appliances and Lights from data_training  (For Calculate Mean Absolute Error)
y_pred=model.predict(X_train)

meanAbsoluteError=mean_absolute_error(y_true,y_pred)
meanAbsoluteError

In [ ]:
y_true

In [ ]:
y_pred

# Using RandomForestRegressor

In [ ]:
regressor=RandomForestRegressor(n_estimators=250,max_features='sqrt')

In [ ]:
regressor.fit(X_train,Y_train)

In [ ]:
y_predictions=regressor.predict(newtestFeature)
y_predictions

# Create Mean Absolute Error (for Random Forest Regressor)

In [ ]:
y_true=np.array(new_copy_data[['Appliances','lights']])

In [ ]:
y_pred=regressor.predict(X_train)

In [ ]:
y_true

In [ ]:
y_pred

In [ ]:
meanAbsoluteErrorRFR=mean_absolute_error(y_true,y_pred)
meanAbsoluteErrorRFR